In [ ]:
pip install optuna

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import pickle
import optuna
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('train_data.csv', index_col=0)
weight = []

In [ ]:
df=df[df["Weight"]!='Not Available']

In [ ]:
for index,row in df.iterrows():
    weight.append(row["Weight"].split()[0])

In [ ]:
df["weight"]=weight

In [ ]:
df = df.astype({"weight": int})

In [ ]:
X = df.drop(["Weight","weight","ID"],axis=1).to_numpy()

In [ ]:
y = df["weight"].to_numpy()

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
def define_model_layers(trial):
    n_layers = 3
    layers = []
    
    in_features = 9
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i),8,128)
        layers.append(out_features)
    
    return layers

In [ ]:
def objective(trial):
    layers = define_model_layers(trial)
    
    lr = trial.suggest_float("lr",1e-5,1e-1,log=True)
    
    mae = []
    
    for train_index, test_index in kf.split(X):
            
            X_train, X_test, y_train, y_test = X[train_index],X[test_index],y[train_index],y[test_index]
            regr = MLPRegressor(hidden_layer_sizes=layers,learning_rate_init=lr,random_state=1, max_iter=1500).fit(X_train, y_train)
            y_pred = regr.predict(X_test)
            mae.append(mean_absolute_error(y_test, y_pred))
            
    acc = sum(mae)/len(mae)
    filename = ""
    for i in layers:
        filename = filename+str(i)+"_"
    filename = filename + str(lr) +".sav"
    
    pickle.dump(regr, open(filename, 'wb'))
    return acc
            

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=100)

print("Best trial:")
trial = study.best_trial

print(" Value: ",trial.value)

print(" Params: ")

for key,value in trial.params.items():
    print("  {}: {}".format(key,value))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [ ]:
regr = MLPRegressor(hidden_layer_sizes=layer,random_state=1, max_iter=1500).fit(X_train, y_train)

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

# Random Forest